<a href="https://colab.research.google.com/github/simulate111/Introduction-to-Human-Language-Technology/blob/main/course_project_Reza2024_FULL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to HLT Project

- Student(s) Name(s): Mohammadreza Akhtari
- Date: June 2024
- Chosen Corpus: imdb
- Contributions (if group project): -

### Corpus information

- Description of the chosen corpus: Large Movie Review Dataset. This is a dataset for binary sentiment classification. A set of 25,000 highly polar movie reviews for training, and 25,000 for testing.
- Paper(s) and other published materials related to the corpus: Maas, Andrew, et al. "Learning word vectors for sentiment analysis." Proceedings of the 49th annual meeting of the association for computational linguistics: Human language technologies. 2011.
- State-of-the-art performance (best published results) on this corpus: Opinion Mining on Movie Reviews Based on Deep Learning Models https://doi.org/10.32604/jai.2023.045617

---

## 1. Setup

In [1]:
# Your code to install and import libraries etc. here
!pip3 install -q transformers[torch] datasets evaluate optuna plotly

---

## 2. Data download and preprocessing

### 2.1. Download the corpus

In [2]:
# Your code to download the corpus here
from pprint import pprint #pprint
import datasets
dset1=datasets.load_dataset("imdb")
pprint(dset1)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [3]:
dset2=dset1.shuffle() #This is never a bad idea, datasets may have ordering to them, which is not what we want
del dset2["unsupervised"] #Delete the unlabeled part of the dataset, we don't need it for anything

In [4]:
pprint(dset2['train'][0]['text'])
print(dset2['train'][0]['label'])

('Okay, first of I hate commenting on this thing but I felt like I had to '
 'stand up for this movie. So many people were bashing on it and I felt like '
 'people who might want to see it should get a second opinion.<br /><br '
 '/>First off, Bend It Like Beckham is not meant to be the most profound movie '
 "of the century. If that's what you're looking for go somewhere else. Just "
 "because it is an independent film does not mean it has to be artsy. It's "
 "supposed to make you feel good and you're supposed to have fun watching it "
 'and those two things are handily accomplished.<br /><br />Secondly, the '
 'acting though not "Halle Berry in Monster\'s Ball" is still good. The movie '
 "doesn't need acting like that honestly so don't look for it. It's a family "
 "movie. If that's what you wanted you wouldn't or shouldn't even be looking "
 'into this movie honestly.<br /><br />Lastly, It has a really cute story. I '
 "think it's thought out well and it's entertaining to watch. I

### 2.2. Preprocessing

In [5]:
dset=dset2
'''
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove URLs
    text = re.sub(r'https?://\S+', '<URL>', text)

    # Remove HTML tags (if any)
    text = re.sub(r'<.*?>', '', text)

    # Remove non-alphanumeric characters except spaces
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Tokenize text
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Join tokens back into a single string
    text = ' '.join(tokens)

    return text

def clean_dataset(dataset):
    cleaned_dataset = []
    for example in dataset:
        cleaned_example = {}
        cleaned_example['text'] = preprocess_text(example['text'])
        cleaned_example['label'] = example['label']
        cleaned_dataset.append(cleaned_example)
    return cleaned_dataset

# Apply to the dataset
dset = clean_dataset(dset2['train'])
'''

"\nimport re\nimport nltk\nfrom nltk.corpus import stopwords\nfrom nltk.tokenize import word_tokenize\n\nnltk.download('punkt')\nnltk.download('stopwords')\n\ndef preprocess_text(text):\n    # Convert to lowercase\n    text = text.lower()\n\n    # Remove URLs\n    text = re.sub(r'https?://\\S+', '<URL>', text)\n\n    # Remove HTML tags (if any)\n    text = re.sub(r'<.*?>', '', text)\n\n    # Remove non-alphanumeric characters except spaces\n    text = re.sub(r'[^a-zA-Z\\s]', '', text)\n\n    # Tokenize text\n    tokens = word_tokenize(text)\n\n    # Remove stopwords\n    stop_words = set(stopwords.words('english'))\n    tokens = [word for word in tokens if word not in stop_words]\n\n    # Join tokens back into a single string\n    text = ' '.join(tokens)\n\n    return text\n\ndef clean_dataset(dataset):\n    cleaned_dataset = []\n    for example in dataset:\n        cleaned_example = {}\n        cleaned_example['text'] = preprocess_text(example['text'])\n        cleaned_example['label'

In [6]:
#Tokenize and map vocabulary
import sklearn.feature_extraction

vectorizer=sklearn.feature_extraction.text.CountVectorizer(binary=True,max_features=20000)

texts=[ex["text"] for ex in dset["train"]] #get a list of all texts from the training data
vectorizer.fit(texts) #"Trains" the vectorizer, i.e. builds its vocabulary

CountVectorizer(binary=True, max_features=20000)

In [7]:
#Building the feature vectors
def vectorize_example(ex):
    vectorized=vectorizer.transform([ex["text"]]) # [...] because the vectorizer expects a list/iterable over inputs, not one input
    non_zero_features=vectorized.nonzero()[1] #.nonzero gives a pair of (rows,columns), we want the columns
    non_zero_features+=1 #feature index 0 will have a special meaning
                         # so let us not produce it by adding +1 to everything
    return {"input_ids":non_zero_features}

vectorized=vectorize_example(dset["train"][0])

In [8]:
print(vectorized)

{'input_ids': array([  307,   369,   423,   775,   863,   891,  1117,  1205,  1555,
        1660,  1703,  1738,  1741,  1834,  1872,  1927,  2300,  2612,
        2717,  3002,  3637,  4431,  4484,  4730,  5400,  5401,  5436,
        5500,  5545,  5951,  6102,  6142,  6307,  6645,  6770,  6787,
        6884,  6925,  6947,  7139,  7385,  7618,  7743,  7783,  8117,
        8148,  8278,  8292,  8311,  8669,  8921,  9084,  9158,  9485,
        9595,  9621,  9875, 10222, 10427, 10453, 10491, 10620, 10624,
       10874, 10958, 11169, 11178, 11365, 11612, 11701, 11746, 11980,
       12188, 12348, 12349, 12391, 12421, 12463, 12492, 12567, 12846,
       13023, 13351, 13819, 14304, 14343, 15489, 15650, 15652, 15678,
       16023, 16026, 16199, 16469, 16541, 16546, 16887, 17019, 17076,
       17413, 17887, 17891, 17945, 17946, 17948, 17961, 17973, 17975,
       17976, 18112, 18462, 18549, 18919, 19112, 19352, 19353, 19412,
       19418, 19507, 19519, 19538, 19548, 19593, 19711, 19805, 19926],
     

In [9]:
# We can map back to vocabulary and check that everything works
# vectorizer.vocabulary_ is a dictionary {key:word, value:idx}

idx2word=dict((i,w) for (w,i) in vectorizer.vocabulary_.items()) #inverse the vocab dictionary
words=[]
for idx in vectorized["input_ids"]:
    words.append(idx2word[idx-1]) ## It is easy to forgot we moved all by +1
pprint(", ".join(words)) #This is now the bag of words representation of the document

('about, accomplished, acting, also, an, and, are, artsy, ball, bashing, be, '
 'because, beckham, bend, berry, big, br, but, can, century, commenting, '
 'culture, cute, deep, does, doesn, don, down, dramatic, else, enjoy, '
 'entertaining, even, family, feel, felt, film, finished, first, for, fun, '
 'get, go, good, had, halle, has, hate, have, honestly, if, in, independent, '
 'into, is, it, just, lastly, life, like, lines, look, looking, make, many, '
 'mean, meant, might, monster, most, movie, need, not, of, off, okay, on, '
 'opinion, or, out, part, people, plot, profound, re, really, scenes, second, '
 'secondly, see, should, shouldn, sit, so, something, somewhere, stand, still, '
 'story, supposed, that, the, thing, things, think, this, those, though, '
 'thought, to, true, two, up, very, want, wanted, watch, watching, well, were, '
 'what, when, who, with, wouldn, you')


In [10]:
# Apply the tokenizer to the whole dataset using .map()
dset_tokenized = dset.map(vectorize_example,num_proc=4)
pprint(dset_tokenized["train"][0])

Map (num_proc=4):   0%|          | 0/25000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/25000 [00:00<?, ? examples/s]

{'input_ids': [307,
               369,
               423,
               775,
               863,
               891,
               1117,
               1205,
               1555,
               1660,
               1703,
               1738,
               1741,
               1834,
               1872,
               1927,
               2300,
               2612,
               2717,
               3002,
               3637,
               4431,
               4484,
               4730,
               5400,
               5401,
               5436,
               5500,
               5545,
               5951,
               6102,
               6142,
               6307,
               6645,
               6770,
               6787,
               6884,
               6925,
               6947,
               7139,
               7385,
               7618,
               7743,
               7783,
               8117,
               8148,
               8278,
               8292

In [11]:
#Padding and Collation (forming a batch)
import torch

def collator(list_of_examples):
    batch={"labels":torch.tensor(list(ex["label"] for ex in list_of_examples))} #this is easy, labels are made into a single tensor
    #the worse bit is now to pad the examples, as they are of different length
    tensors=[]
    max_len=max(len(example["input_ids"]) for example in list_of_examples) #this is the longest example in the batch
    #everything needs to be padded to fit in length the longest example
    #(so we can build a single tensor out of it)
    for example in list_of_examples:
        ids=torch.tensor(example["input_ids"]) #pick the input ids
        # pad(what,(from_left, from_right)) <- this is how we call the stock pad function
        padded=torch.nn.functional.pad(ids,(0,max_len-ids.shape[0])) #pad by max - current length, pads with zero by default
        tensors.append(padded) #accumulated the padded ids
    batch["input_ids"]=torch.vstack(tensors) #now that we have all of them the same length, a simple vstack() stacks them up
    return batch #...and that's all there is to it

In [12]:
#Build a batch from 2 examples, with padding
batch=collator([dset_tokenized["train"][2],dset_tokenized["train"][7]])
print("Shape of labels:",batch["labels"].shape)
print("Shape of input_ids:",batch["input_ids"].shape)
pprint(batch["labels"])
pprint(batch["input_ids"])

Shape of labels: torch.Size([2])
Shape of input_ids: torch.Size([2, 442])
tensor([0, 0])
tensor([[   10,    17,   177,   250,   307,   318,   362,   423,   424,   434,
           440,   472,   593,   652,   731,   765,   775,   787,   863,   891,
           930,   963,  1011,  1013,  1048,  1117,  1209,  1214,  1291,  1302,
          1318,  1326,  1337,  1411,  1433,  1494,  1516,  1603,  1654,  1661,
          1703,  1738,  1757,  1900,  1996,  2045,  2101,  2148,  2166,  2300,
          2633,  2684,  2697,  2748,  2916,  2935,  3002,  3076,  3162,  3317,
          3679,  4054,  4127,  4461,  4637,  4694,  4695,  4778,  4912,  4934,
          4947,  5093,  5165,  5169,  5171,  5255,  5377,  5400,  5409,  5436,
          5500,  5714,  5728,  5753,  6127,  6142,  6249,  6307,  6316,  6324,
          6589,  6668,  6770,  6884,  6911,  6960,  7132,  7166,  7344,  7403,
          7468,  7618,  7620,  7621,  7671,  7783,  7998,  8078,  8278,  8311,
          8339,  8340,  8342,  8474,  8543

---

## 3. Machine learning model

### 3.1. Model training

In [13]:
#Build the MLP model
import torch
import transformers

# A model wants a config, I can simply inherit from the base
# class for pretrained configs
class MLPConfig(transformers.PretrainedConfig):
    pass

# This is the model
class MLP(transformers.PreTrainedModel):

    config_class=MLPConfig

    # In the initialization method, one instantiates the layers
    # these will be, for the most part the trained parameters of the model
    def __init__(self,config):
        super().__init__(config)
        self.vocab_size=config.vocab_size #embedding matrix row count
        # Build and initialize embedding of vocab size +1 x hidden size (+1 because of the padding index 0!)
        self.embedding=torch.nn.Embedding(num_embeddings=self.vocab_size+1,embedding_dim=config.hidden_size,padding_idx=0)
        # Normally you would not initialize these yourself, but I have my reasons here ;)
        torch.nn.init.uniform_(self.embedding.weight.data,-0.001,0.001) #initialize the embeddings with small random values
        # Note! This is quite clever and keeps the embedding for 0, the padding, pure zeros
        # This takes care of the lower half of the network, now the upper half
        # Output layer: hidden size x output size
        self.output=torch.nn.Linear(in_features=config.hidden_size,out_features=config.nlabels)
        # Now we have the parameters of the model

    def forward(self,input_ids,labels=None): #nevermind the attention_mask, its time will come, data collator insists on adding it
        #sum up the embeddings of the items
        embedded=self.embedding(input_ids) #(batch,ids)->(batch,ids,embedding_dim)
        # Since the Embedding keeps the first row of the matrix pure zeros, we don't need to worry about the padding
        # so next we sum the embeddings across the word dimension
        # (batch,ids,embedding_dim) -> (batch,embedding_dim)
        embedded_summed=torch.sum(embedded,dim=1)
        logits=self.output(embedded_summed)

        # We have labels, so we ought to calculate the loss
        if labels is not None:
            loss=torch.nn.CrossEntropyLoss() #This loss is meant for classification, so let's use it
            # You run it as loss(model_output,correct_labels)
            return (loss(logits,labels),logits)
        else:
            # No labels, so just return the logits
            return (logits,)

In [14]:
# Configure the model:
mlp_config=MLPConfig(vocab_size=len(vectorizer.vocabulary_),hidden_size=1,nlabels=2)

In [15]:
#Make a model
mlp=MLP(mlp_config)
fake_batch=collator([dset_tokenized["train"][0],dset_tokenized["train"][1]])
mlp(**fake_batch) #** expands input_ids and labels as parameters of the call

(tensor(1.2186, grad_fn=<NllLossBackward0>),
 tensor([[ 0.2972, -0.5967],
         [ 0.2734, -0.5686]], grad_fn=<AddmmBackward0>))

In [16]:
# Set training arguments
# their names are mostly self-explanatory
trainer_args = transformers.TrainingArguments(
    "mlp_checkpoints", #save checkpoints here
    evaluation_strategy="steps",
    logging_strategy="steps",
    eval_steps=500,
    logging_steps=500,
    learning_rate=1e-4, #learning rate of the gradient descent
    max_steps=40000, #20000,
    load_best_model_at_end=True,
    per_device_train_batch_size=128
)

pprint(trainer_args)

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_steps=500,
eval_strategy=steps,
evaluation_strategy=steps,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [17]:
# A metric for evaluating performance
import numpy as np
import evaluate

accuracy = evaluate.load("accuracy")

def compute_accuracy(outputs_and_labels):
    outputs, labels = outputs_and_labels
    predictions = np.argmax(outputs, axis=-1) #pick the index of the "winning" label
    return accuracy.compute(predictions=predictions, references=labels)

In [18]:
# Make a new model
mlp = MLP(mlp_config)


# Argument gives the number of steps of patience before early stopping
# i.e. training is stopped when the evaluation loss fails to improve
# certain number of times
early_stopping = transformers.EarlyStoppingCallback(5)

trainer = transformers.Trainer(
    model=mlp,
    args=trainer_args,
    train_dataset=dset_tokenized["train"],
    eval_dataset=dset_tokenized["test"].select(range(1000)), #make a smaller subset to evaluate on
    compute_metrics=compute_accuracy,
    data_collator=collator,
    callbacks=[early_stopping]
)

# FINALLY!
trainer.train()

max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss,Validation Loss,Accuracy
500,0.594100,0.512656,0.810000
1000,0.431700,0.420667,0.865000
1500,0.354500,0.370870,0.873000
2000,0.306500,0.338811,0.879000
2500,0.273200,0.317328,0.886000
3000,0.249200,0.302008,0.889000
3500,0.227100,0.290999,0.887000
4000,0.211000,0.282226,0.887000
4500,0.196900,0.276771,0.892000
5000,0.183800,0.271737,0.893000


TrainOutput(global_step=9500, training_loss=0.22557449140046773, metrics={'train_runtime': 179.0629, 'train_samples_per_second': 28593.299, 'train_steps_per_second': 223.385, 'total_flos': 12420685248.0, 'train_loss': 0.22557449140046773, 'epoch': 48.46938775510204})

In [19]:
#Evaluation
eval_results = trainer.evaluate(dset_tokenized["test"])

print(eval_results)

{'eval_loss': 0.2811674475669861, 'eval_accuracy': 0.88688, 'eval_runtime': 6.4718, 'eval_samples_per_second': 3862.918, 'eval_steps_per_second': 482.865, 'epoch': 48.46938775510204}


### 3.2 Hyperparameter optimization

### 3.3. Evaluation on test set

In [20]:
import optuna

def objective(trial):
    # Define the search space for hyperparameters
    learning_rate = trial.suggest_float("learning_rate", 1e-6, 1e-1, log=True)
    batch_size = trial.suggest_categorical("batch_size", [16, 64, 128, 256])

    trainer_args = transformers.TrainingArguments(
        "mlp_checkpoints", #save checkpoints here
        evaluation_strategy="steps",
        logging_strategy="steps",
        eval_steps=500,
        logging_steps=500,
        learning_rate=learning_rate, #learning rate of the gradient descent
        max_steps=40000, #10000, #I will keep this small, no time to wait too long :)
        load_best_model_at_end=True,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size
    )

    mlp = MLP(mlp_config)

    trainer = transformers.Trainer(
        model=mlp,
        args=trainer_args,
        train_dataset=dset_tokenized["train"],
        eval_dataset=dset_tokenized["test"], #dset_tokenized["test"].select(range(1000)), #make a smaller subset to evaluate on
        compute_metrics=compute_accuracy,
        data_collator=collator,
        callbacks=[early_stopping]
    )

    # Train the model and get the best validation loss
    trainer.train()
    eval_results = trainer.evaluate(eval_dataset=dset_tokenized["test"]) #trainer.evaluate()
    return eval_results["eval_accuracy"] #let's try to maximize accuracy

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=7) #I will keep this small, no time to wait :)

[I 2024-06-16 13:27:47,915] A new study created in memory with name: no-name-c6113b77-9620-40ae-891c-c9d03369f71f
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss,Validation Loss,Accuracy
500,0.426900,0.311226,0.873000
1000,0.283600,0.305410,0.874520
1500,0.284700,0.292088,0.882840
2000,0.158600,0.324944,0.878640
2500,0.167100,0.338984,0.874960
3000,0.170000,0.348777,0.871280
3500,0.107800,0.425744,0.864480
4000,0.093500,0.460432,0.863840


[I 2024-06-16 13:28:53,214] Trial 0 finished with value: 0.88284 and parameters: {'learning_rate': 0.0028405788477162846, 'batch_size': 16}. Best is trial 0 with value: 0.88284.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss,Validation Loss,Accuracy
500,0.736900,0.708646,0.508840
1000,0.686900,0.677928,0.551680
1500,0.660500,0.653874,0.606040
2000,0.632300,0.627884,0.673240
2500,0.601300,0.599800,0.736600
3000,0.572500,0.571816,0.778400
3500,0.541000,0.544707,0.807240
4000,0.511800,0.519361,0.826800
4500,0.486600,0.496391,0.838280
5000,0.460100,0.475587,0.845960


[I 2024-06-16 13:40:38,683] Trial 1 finished with value: 0.88984 and parameters: {'learning_rate': 4.671898970848419e-05, 'batch_size': 64}. Best is trial 1 with value: 0.88984.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


...

[I 2024-06-16 14:23:24,713] Trial 4 finished with value: 0.87576 and parameters: {'learning_rate': 0.006035009622352109, 'batch_size': 64}. Best is trial 1 with value: 0.88984.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss,Validation Loss,Accuracy
500,0.696100,0.693874,0.500000
1000,0.690800,0.688223,0.512160
1500,0.684200,0.681820,0.569040
2000,0.676800,0.674966,0.644560
2500,0.669000,0.667722,0.708440
3000,0.660500,0.660001,0.744600
3500,0.651300,0.651857,0.766160
4000,0.642000,0.643396,0.781040
4500,0.632100,0.634715,0.790600
5000,0.622200,0.625906,0.797320


[I 2024-06-16 14:49:16,135] Trial 5 finished with value: 0.86604 and parameters: {'learning_rate': 1.4724660907331185e-05, 'batch_size': 256}. Best is trial 1 with value: 0.88984.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss,Validation Loss,Accuracy
500,0.338900,0.282675,0.886200
1000,0.154500,0.328608,0.876320
1500,0.094900,0.392650,0.869720
2000,0.056400,0.480701,0.863560
2500,0.029400,0.572586,0.859320
3000,0.017600,0.672226,0.854560


[I 2024-06-16 14:50:13,327] Trial 6 finished with value: 0.8862 and parameters: {'learning_rate': 0.0026832271576898293, 'batch_size': 64}. Best is trial 1 with value: 0.88984.


In [21]:
# Print the best trial's hyperparameters and objective value
best_trial = study.best_trial
print(f"Best trial (number {best_trial.number}):")
print(f"  Value: {best_trial.value}")
print(f"  Params: {best_trial.params}")

# Print all trials' hyperparameters and objective values
print("\nAll trials:")
for trial in study.trials:
    print(f"  Trial {trial.number}:")
    print(f"    Value: {trial.value}")
    print(f"    Params: {trial.params}")

Best trial (number 1):
  Value: 0.88984
  Params: {'learning_rate': 4.671898970848419e-05, 'batch_size': 64}

All trials:
  Trial 0:
    Value: 0.88284
    Params: {'learning_rate': 0.0028405788477162846, 'batch_size': 16}
  Trial 1:
    Value: 0.88984
    Params: {'learning_rate': 4.671898970848419e-05, 'batch_size': 64}
  Trial 2:
    Value: 0.79016
    Params: {'learning_rate': 3.400610504968987e-06, 'batch_size': 256}
  Trial 3:
    Value: 0.79576
    Params: {'learning_rate': 7.079172599523785e-06, 'batch_size': 128}
  Trial 4:
    Value: 0.87576
    Params: {'learning_rate': 0.006035009622352109, 'batch_size': 64}
  Trial 5:
    Value: 0.86604
    Params: {'learning_rate': 1.4724660907331185e-05, 'batch_size': 256}
  Trial 6:
    Value: 0.8862
    Params: {'learning_rate': 0.0026832271576898293, 'batch_size': 64}


In [22]:
import optuna.visualization as vis
fig = vis.plot_parallel_coordinate(study)
fig.show()


In [23]:
fig = vis.plot_slice(study)
fig.show()

In [24]:
fig = vis.plot_contour(study, params=['learning_rate', 'batch_size'])
fig.show()

In [25]:
fig = vis.plot_optimization_history(study)
fig.show()

---

## 4. Results and summary

### 4.1 Corpus insights

Large Movie Review Dataset. This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. The model predict whether a comment is positive or negative based on the training set, which is evaluated on the test set.
Analysing comments is very importnat to due the existense of abundance comments, which is not possibel to be handled by human. THe automatic evaluation of comments allows us to evaluate and rate each case based on the comments and opinions automatically in a real-time. As the language is processing over time, it is of great importance to develop the model and increase the parameters over time as the available comments, words, expressions, and advanced technology increases to achieve better results.

### 4.2 Results

My model inspired by the exercise 6 in the course to facilitate th programming and avoid mistakes. The model was able to achieve the accuracy of 89% ('learning_rate': 0.000828301723840474, 'batch_size': 128) while in https://doi.org/10.32604/jai.2023.045617, an impressive 93.74% accuracy on the IMDb Dataset has been avhievable recently using Extra-Long Neural Network (XLNet).
THe higher values achieve almost with lower learning rate but higher batch size huperparameters.

In another study in 2020, 91% was achievable in Sentiment analysis on IMDB using lexicon and neural networks.(https://doi.org/10.1007/s42452-019-1926-x). Almost the same value was achievable in the very new publication in 2024 of What's Next?: Exploring Machine Learning-Based Approaches to Content Suggestions Using IMDb Movie Reviews (https://doi.org10.1109/ICEIC61013.2024.10457130). HOwever, achieving an impressive 93.74% accuracy on the IMDb Dataset is seeable in Opinion Mining on Movie Reviews Based on Deep Learning Models (https://doi.org/10.32604/jai.2023.045617).

### 4.3 Relation to state of the art

As could be seen, my performance is close to the state of the art results. However, achieving 93% accuracy is very difficult which was able to be reached by using extra-long neural network requiring a huge amount of processing and time (XLNet is an advanced method that combines the advantages of Autoregressive and Autoencoding methods through a technique called permutation language modeling. The neural architecture of XLNet is specifically designed to perform better at Autoregressive tasks, such as Transformer-XL and the carefully crafted two-stream attention mechanism.)."Although XLNet
performed better in many ways, its longer training time may be an issue in circumstances when we need faster results but have limited resources or time to train the model"
However, my model seems to have an acceptable accuracy which is very usefull and applicable for general use unless the sufficient timea and processing equipments are available or needed.
---

## 5. Bonus Task (optional)

### 5.1. Annotating out-of-domain documents

(Briefly describe the chosen out-of-domain documents)

(Briefly describe the process of annotation)

### 5.2 Conversion into dataset

In [26]:
# Your code to convert the annotations into a dataset here

### 5.3. Model evaluation on out-of-domain test set

In [27]:
# Your code to evaluate the model on the out-of-domain test set here

### 5.4 Bonus task results

(Present the results of the evaluation on the out-of-domain test set)

### 5.5. Annotated data

In [28]:
# Include your annotated out-of-domain data here